In [1]:
import numpy as np
import csv
import pandas as pd
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from datetime import date, datetime
from copy import deepcopy
from IPython.display import clear_output


from utils import read_basic_dataset

from features import get_month_virus_share, create_trap_distance_matrix, get_nearest_trap, get_nearest_trap_list, \
    add_multirows

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
fold_count = 5
seed = 1337
train_verbose = 0

# Load Basic Data

In [3]:
training_features, training_target, test_features = read_basic_dataset()

In [4]:
trap_records = pd.concat([training_features[['Trap', 'Latitude', 'Longitude']], 
                          test_features[['Trap', 'Latitude', 'Longitude']]])
trap_distance_matrix = create_trap_distance_matrix(trap_records)
#trap_distance_matrix.info()

In [5]:
test_features.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,...,10dtmin_min,10dtavg_avg,10dtavg_max,10dtavg_min,10dpcp_tot,10ddwp_avg,10dprs_avg,year,month,week
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,53,71.090909,80.0,62.0,2.654,60.363636,29.105455,2008,6,24
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,53,71.090909,80.0,62.0,2.654,60.363636,29.105455,2008,6,24
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,53,71.090909,80.0,62.0,2.654,60.363636,29.105455,2008,6,24
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,53,71.090909,80.0,62.0,2.654,60.363636,29.105455,2008,6,24
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,53,71.090909,80.0,62.0,2.654,60.363636,29.105455,2008,6,24


In [6]:
virus_per_trap_species, virus_per_trap, virus_per_species_overall = get_month_virus_share(training_features)

# Data Preparation

In [7]:
overall_rate = training_features['WnvPresent'].sum()/training_features['NumMosquitos'].sum()
def virusshare_best_source(row):
    if not pd.isnull(row['virusshare_ts']):
        return row['virusshare_ts']
    elif not (pd.isnull(row['virusshare_t']) or pd.isnull(row['species_virus'])):
        return row['virusshare_t'] * row['species_virus'] / overall_rate
    elif not pd.isnull(row['species_virus']):
        return row['species_virus']
    else:
        return 0

training_features = pd.merge(how='left', left=training_features, right=virus_per_trap_species, 
                             left_on=['month', 'Trap', 'Species'], right_index=True)
training_features['virusshare'] = training_features.apply(lambda _: _['virusshare_ts'], axis=1)
training_features['species_virus'] = training_features.Species.\
    apply(lambda _: virus_per_species_overall.loc[_]['virusshare_s'] if _ in virus_per_species_overall.index else -1)
#cols_to_use = test_features.columns.difference(virus_share_df.columns)
test_features = pd.merge(how='left', left=test_features, right=virus_per_trap_species, 
                             left_on=['month', 'Trap', 'Species'], right_index=True)
test_features = pd.merge(how='left', left=test_features, right=virus_per_trap, 
                             left_on=['month', 'Trap'], right_index=True)
test_features['species_virus'] = test_features.Species.\
    apply(lambda _: virus_per_species_overall.loc[_]['virusshare_s'] 
          if _ in virus_per_species_overall.index else -1)
test_features['virusshare'] = test_features.apply(virusshare_best_source, axis=1)

#test_features.head()


In [8]:
training_features = add_multirows(training_features)
test_features = add_multirows(test_features)

In [9]:
keep_features = ['week', 'Latitude', 'Longitude', 
                 'Tmax', 'Tmin', 'Tavg', 'DewPoint', 'StnPressure', 'PrecipTotal',
                 '', '10dtmin_min', '10dtavg_avg', '10dpcp_tot', '10ddwp_avg', '10dprs_avg']
drop_features = ['AddressAccuracy','AddressNumberAndStreet','Address', 'Block', 'Date', 
                 'Heat', 'Cool', 'Sunrise', 'Sunset','Depth','Water1','SeaLevel', 'SnowFall', 'CodeSum', 
                 'Depart', 'WetBulb', 'ResultSpeed', 'ResultDir', 'AvgSpeed', 
                 'month', 'Species', 'station','Street', 'Trap', 'Station', 'ddate', 'year',
                 'NumMosquitos', 'WnvPresent', 'Id',
                 'virusshare_ts', 'virusshare_t', 'location_distance',
                 #'10dtmax_max',
                 '10dtmax_min', 
                 '10dtmax_avg', 
                 '10dtavg_max', 
                 #'10dtavg_avg', 
                 #'10dtavg_min', 
                 '10dtmin_max', 
                 '10dtmin_avg',
                 #'10dtmin_min', 
                 #'10dpcp_tot', 
                 #'10ddwp_avg', 
                 #'10dprs_avg'
                ]
training_features_input = training_features.copy()
test_features_input = test_features.copy()
for drop_feature in drop_features:
    if drop_feature in training_features_input.columns:
        training_features_input = training_features_input.drop([drop_feature], axis=1)
    else: 
        print('Feature {} not among training features'.format(drop_feature))
    if drop_feature in test_features_input.columns:
        test_features_input = test_features_input.drop([drop_feature], axis=1)
    else: 
        print('Feature {} not among testing features'.format(drop_feature))



Feature NumMosquitos not among testing features
Feature WnvPresent not among testing features
Feature Id not among training features
Feature virusshare_t not among training features
Feature location_distance not among training features
Feature location_distance not among testing features


In [10]:
np.random.seed(seed)
shuffle = np.arange(len(training_features_input))
np.random.shuffle(shuffle)
training_target_input = training_target.iloc[shuffle]
training_features_input = training_features_input.iloc[shuffle]



In [11]:
scaler = StandardScaler()
scaler.fit(training_features_input)
training_feature_array = scaler.transform(training_features_input)
training_target_array = np.asarray(training_target_input)
test_feature_array = scaler.transform(test_features_input.fillna(0))



# Model Definition

In [12]:
def train_model(fold_cnt, feature_array, target_array, model_generator, fitting_function):
    folds = KFold(len(target_array), fold_count)
    mean_auroc_valid = 0.
    mean_auroc_train = 0
    target_array_categorical = np_utils.to_categorical(target_array)
    trained_models = list()
    for i, (train, valid) in enumerate(folds):
        print('Fold', i)
        X_train = feature_array[train]
        X_valid = feature_array[valid]
        Y_train = target_array_categorical[train]
        y_train = target_array[train]
        Y_valid = target_array_categorical[valid]
        y_valid = target_array[valid]
        foldmodel = model_generator()
        train_and_valid_data = (X_train, Y_train, y_train, X_valid, Y_valid, y_valid)
        fitting_function(foldmodel, train_and_valid_data)
        trained_models.append(foldmodel)
        valid_preds = foldmodel.predict_proba(X_valid)
        training_preds = foldmodel.predict_proba(X_train)
        roc_valid = metrics.roc_auc_score(y_valid, valid_preds[:, 1])
        roc_train = metrics.roc_auc_score(y_train, training_preds[:, 1])
        #print("ROC: {} training, {} validation".format(roc_train, roc_valid))
        mean_auroc_train += roc_train
        mean_auroc_valid += roc_valid
            
    print('Average ROC:', mean_auroc_train/fold_count, mean_auroc_valid/fold_count)
    return trained_models

    

## XGB

In [13]:
#xgb = XGBClassifier()
#xgb.fit(training_feature_array, training_target_array.ravel())
#xgb.predict_proba(np.array(test_feature_array))

def xgb_model_builder(xgb_model_dict=None):
    if not xgb_model_dict:
        xgb_model_dict={'n_estimators': 300,
                        'max_depth': 3,
                        'reg_alpha':0.01,
                        'seed':seed}
    return XGBClassifier(**xgb_model_dict)

def xgb_fitting_function(xgb_model, tvd):
    xgb_model.fit(tvd[0], tvd[2].ravel())
    
xgbms = train_model(fold_count, training_feature_array, training_target_array, xgb_model_builder, xgb_fitting_function)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Average ROC: 0.964825085173 0.916808799635


In [14]:
xgb_prediction_array = [xgb.predict_proba(np.nan_to_num(test_feature_array))[:,1] 
                        for xgb in xgbms]
bagged_xgb_prediction = np.mean(np.array(xgb_prediction_array), axis=0)
bagged_xgb_prediction

array([ 0.01533597,  0.00656668,  0.01593937, ...,  0.00010511,
        0.00010511,  0.00010511], dtype=float32)

In [15]:
feature_importance_array = np.mean(np.array([xgb.feature_importances_ for xgb in xgbms]), axis=0)
feature_importance_df = pd.DataFrame(list(zip(test_features_input.columns, feature_importance_array)))
feature_importance_df.columns = ['Feature', 'Importance']
feature_importance_df.set_index('Feature')

,Importance
Feature,
Latitude,0.120278
Longitude,0.110495
Tmax,0.043694
Tmin,0.013307
Tavg,0.025495
DewPoint,0.023814
PrecipTotal,0.017491
StnPressure,0.032762
10dtmax_max,0.031814


## Dense Neural Network

In [16]:
model_dict = {
    'loss': 'categorical_crossentropy',
    'optimizer': 'adadelta',
    'layers': [{'nodecount': 30, 'activation': 'relu', 'dropout': 0.5},
               {'nodecount': 20, 'activation': 'relu', 'dropout': 0.25},
               {'nodecount': 10, 'activation': 'relu', 'dropout': 0.125}],
    'dimension_out': 2
}

In [17]:
def build_model(model_dict):
    model = Sequential()
    input_dim = model_dict['dimension_input']
    for layer in model_dict['layers']:
        model.add(Dense(layer['nodecount'], input_dim=input_dim))
        model.add(Activation(layer['activation']))
        model.add(Dropout(layer['dropout']))
        input_dim = layer['nodecount']

    model.add(Dense(model_dict['dimension_output']))
    model.add(Activation('softmax'))

    model.compile(loss=model_dict['loss'], optimizer=model_dict['optimizer'])
    return model

model_dict['dimension_input'] = training_feature_array.shape[1]
model_dict['dimension_output'] = len(np.unique(training_target_array))

def keras_model_builder():
    return build_model(model_dict)
def keras_fit(keras_model, tvd):
    keras_model.fit(tvd[0], tvd[1], epochs=50, batch_size=32, validation_data=(tvd[3], tvd[4]), verbose=train_verbose)

fnn = keras_model_builder()
fnnms = train_model(fold_count, training_feature_array, training_target_array, keras_model_builder, keras_fit)
#keras_fit(fnn, )
#FNN = train_model(4, train_init_array, train_target_id, keras_model_builder, keras_fit)

Fold 0
8405/8405 [==============================] - 0s     
Fold 3
5888/8405 [====================>.........] - ETA: 0sAverage ROC: 0.906141456636 0.894112187451


In [18]:
fnn_prediction_array = [fnn.predict_proba(np.nan_to_num(test_feature_array))[:,1] for fnn in fnnms]
bagged_fnn_prediction = np.mean(np.array(fnn_prediction_array), axis=0)

116288/116293 [============================>.] - ETA: 0s

In [20]:
test_features['WnvPresent'] = bagged_xgb_prediction
export_df = test_features[['Id', 'WnvPresent']]
export_df.to_csv('multirow.csv', index=False , quotechar='"')